<a href="https://colab.research.google.com/github/anacanassa/ChatBot_Meteo/blob/main/ChatBot_Meteo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalação de biblioteca e configuracão da chave da API

In [1]:
!pip install -q -U google-genai

from google import genai
from google.colab import userdata

api_key = userdata.get("GEMINI_API_KEY")
if not api_key:
    raise ValueError("SECRET GEMINI_API_KEY não encontrado!")

client = genai.Client(api_key=api_key)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 703.4/703.4 kB 11.8 MB/s eta 0:00:00


Função de Limpeza do JSON recebido pela resposta do Gemini

In [2]:
import json
import re

def clean_gemini_json(text: str) -> str:
    """Remove markdown, espaços e extrai apenas o JSON válido."""
    if not text:
        return text

    # Remove blocos ```json e ``` simples
    text = text.replace("```json", "").replace("```", "")

    # Remove espaços extras no início e fim
    text = text.strip()

    # Se houver mais de um JSON no texto, pegar apenas o primeiro {...}
    match = re.search(r"\{.*\}", text, re.DOTALL)
    if match:
        text = match.group(0)

    return text


Função de tratamento do Formato de Data recebido pelo Gemini

In [3]:
from datetime import date, timedelta, datetime
import unicodedata
import re

def _normalize_token(s: str) -> str:
    """Normaliza: remove acentos, pontuação, lower e trim."""
    if not isinstance(s, str):
        return ""
    # remove acentos
    s = unicodedata.normalize("NFKD", s).encode("ASCII", "ignore").decode("utf-8")
    # lower e strip
    s = s.lower().strip()
    # remove pontuação (ex.: "atual.", "hoje!")
    s = re.sub(r"[^\w\s]", "", s)
    # normaliza espaços múltiplos
    s = re.sub(r"\s+", " ", s)
    return s

def parse_date(date_str: str):
    """
    Converte uma string para datetime.date:
    - aceita tokens relativos: 'hoje', 'atual', 'agora', 'amanha', 'ontem' (variações com/sem acento)
    - aceita ISO YYYY-MM-DD
    - aceita DD/MM/YYYY e DD-MM-YYYY
    - retorna datetime.date ou None se não conseguir interpretar
    """
    if date_str is None:
        return None

    token = _normalize_token(date_str)

    today = date.today()

    if token in {"hoje", "atual", "agora", "today"}:
        return today
    if token in {"amanha", "amanhã", "amanha", "tomorrow"}:
        return today + timedelta(days=1)
    if token in {"ontem", "yesterday"}:
        return today - timedelta(days=1)

    # tenta ISO YYYY-MM-DD
    try:
        return datetime.fromisoformat(token).date()
    except Exception:
        pass

    # tenta DD/MM/YYYY e DD-MM-YYYY
    for fmt in ("%d/%m/%Y", "%d-%m-%Y"):
        try:
            return datetime.strptime(date_str.strip(), fmt).date()
        except Exception:
            pass

    # Não conseguiu interpretar
    return "Não entendi"

#print(parse_date("atual"))


Função de pergunta ao Gemnini

In [4]:
def ask_gemini_extract_params(user_question: str):
    try:
        prompt = f"""
        Extraia em JSON os seguintes parâmetros da pergunta do usuário:
        - city: nome da cidade
        - date: data em formato YYYY-MM-DD ou em data relativa (hoje, amanha, ontem, agora, atual)
        - info: tipo de informação climática (temperature ou weathercode)

        Pergunta: "{user_question}"

        Responda SOMENTE com um JSON.
        """

        response = client.models.generate_content(
            model="gemini-2.5-flash",
            contents=prompt
        )

        text = response.text

        cleaned = clean_gemini_json(text)

        params = json.loads(cleaned)
        return params

    except Exception as e:
        print(f"Erro ao chamar Gemini: {e}")
        return None

##print(ask_gemini_extract_params("Qual o clima atual de São Paulo?"))

Função de consulta das Coordenadas da Cidade pesquisada

In [5]:
import requests

def get_city_coords(city_name: str):
    """
    Busca latitude e longitude de uma cidade usando Open-Meteo Geocoding API.
    """
    try:
        url = f"https://geocoding-api.open-meteo.com/v1/search?name={city_name}&count=1&language=pt&format=json"
        response = requests.get(url, timeout=10)

        if response.status_code != 200:
            print("Erro na API de geolocalização:", response.status_code)
            return None

        data = response.json()

        # Nenhum resultado encontrado
        if "results" not in data or len(data["results"]) == 0:
            return None

        # Primeiro resultado da API
        result = data["results"][0]

        lat = result.get("latitude")
        lon = result.get("longitude")

        if lat is None or lon is None:
            return None

        return lat, lon

    except Exception as e:
        print(f"Erro ao consultar coordenadas: {e}")
        return None


Função de consulta do Clima no Open Meteo

In [6]:
weather_map = {
0: "Céu limpo",
1: "Predominantemente claro",
2: "Parcialmente nublado",
3: "Nublado",
61: "Chuva fraca",
63: "Chuva moderada",
65: "Chuva forte",
95: "Tempestade"
}

import requests
from datetime import datetime

def get_weather(lat, lon, target_date):
    """
    Consulta a API Open-Meteo e retorna:
    - temperatura máxima (float)
    - condição climática traduzida (string)
    """

    try:
        url = (
            "https://api.open-meteo.com/v1/forecast?"
            f"latitude={lat}&longitude={lon}"
            "&daily=temperature_2m_max,weathercode"
            "&timezone=America/Sao_Paulo"
        )

        response = requests.get(url)
        response.raise_for_status()
        data = response.json()

        dates = data["daily"]["time"]
        temps = data["daily"]["temperature_2m_max"]
        codes = data["daily"]["weathercode"]

        target_str = target_date.isoformat()

        if target_str not in dates:
            return None, None

        idx = dates.index(target_str)
        temp = temps[idx]
        code = codes[idx]

        condition = weather_map.get(code, "Condição não disponível")

        return temp, condition

    except Exception as e:
        print("Erro no get_weather:", e)
        return None, None




Função do Chatbot com as Chamadas Gerais das Funões

In [7]:

def chatbot_full(user_question: str):
    params = ask_gemini_extract_params(user_question)


    if not params:
        return "Não consegui entender sua pergunta. Tente reformular."

    city = (params.get("city") or "").strip().title()
    date_str = params.get("date") or ""
    info = params.get("info") or ""

    if not city:
        return "Pergunte novamente informando sua Cidade"

    coords = get_city_coords(city)
    if not coords:
        return f"Não consegui encontrar coordenadas para '{city}'."

    lat, lon = coords

    target_date = parse_date(date_str)
    if not target_date:
        return f"Data inválida: {date_str}"

    # Consulta Open-Meteo
    temp, cond = get_weather(lat, lon, target_date)
    if temp is None:
        return f"Sem dados climáticos para {city} em {target_date}."

    if info == "temperature":
        return f"A temperatura máxima em {city} em {target_date} será de {temp}°C."
    else:
        return f"A condição climática em {city} em {target_date} será: {cond}."


#print(chatbot_full("Qual o clima atual de São Paulo?"))


In [8]:
print("🌦️ Chatbot Meteorológico iniciado! Digite 'sair' para encerrar.\n")

while True:
    question = input("Você: ")
    if question.lower() == "sair":
        break

    answer = chatbot_full(question)
    print("Bot:", answer, "\n")

🌦️ Chatbot Meteorológico iniciado! Digite 'sair' para encerrar.

Você: Qual a temperatura para São Paulo hoje?
Bot: A temperatura máxima em São Paulo em 2025-12-12 será de 31.6°C. 

Você: Vai fazer sol no Rio de janeiro hoje?
Bot: A condição climática em Rio De Janeiro em 2025-12-12 será: Nublado. 

Você: sair
